<a href="https://colab.research.google.com/github/tarunkalva14/Pyspark-with-Jyupter-/blob/main/Example_of_Pyspark_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [4]:
training = spark.read.csv('/test1.csv',header=True,inferSchema=True)

In [5]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [8]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [9]:
type(training)

pyspark.sql.dataframe.DataFrame

In [10]:
from pyspark.ml.feature import VectorAssembler

In [11]:
feautreassembler=VectorAssembler(inputCols=['age','Experience'],outputCol='Independent Features')

In [12]:
output=feautreassembler.transform(training)

In [13]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [14]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [17]:
finalized_data=output.select('Independent Features','Salary')

In [18]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [22]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features',labelCol='Salary')
regressor=regressor.fit(train_data)

In [23]:
regressor.coefficients

DenseVector([-176.3341, 1320.1856])

In [24]:
regressor.intercept

19812.064965197555

In [25]:
pred_results=regressor.evaluate(test_data)

In [26]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|17429.234338747123|
|         [31.0,10.0]| 30000|27547.563805104444|
+--------------------+------+------------------+



In [27]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(2440.8352668213392, 5957811.381290981)